# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
from sqlalchemy import create_engine
import pandas
import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///dis_data.db')
con = engine.connect()
df = pandas.read_sql_table('Data_Clean', con)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10170 entries, 0 to 10169
Data columns (total 40 columns):
id                        10170 non-null int64
message                   10170 non-null object
original                  10170 non-null object
genre                     10170 non-null object
related                   10170 non-null int64
request                   10170 non-null int64
offer                     10170 non-null int64
aid_related               10170 non-null int64
medical_help              10170 non-null int64
medical_products          10170 non-null int64
search_and_rescue         10170 non-null int64
security                  10170 non-null int64
military                  10170 non-null int64
child_alone               10170 non-null int64
water                     10170 non-null int64
food                      10170 non-null int64
shelter                   10170 non-null int64
clothing                  10170 non-null int64
money                     10170 non-null i

In [4]:
# Create Feature and Label Vectors

#Create Labels
ls = df.columns[4:]
Y = None
for el in ls:
    Y = pandas.concat([Y, df[el]], axis = 1)
Y = Y.dropna()
Y["related"].replace({2: 0}, inplace=True)

# Create Features
X = df['message'] 
print(X.shape, Y.shape)

(10170,) (10170, 36)


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    lemmed = [WordNetLemmatizer().lemmatize(w).strip() for w in words]
    return lemmed

In [6]:
# Test the tokenize function
tokenize("I wish this were a simpler sentence, but it's not")

['wish', 'simpler', 'sentence']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
# Create the Pipeline
pipeline = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)), 
                    ('tfidf', TfidfTransformer()),
                    ('clf', RandomForestClassifier())])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
cv = GridSearchCV(pipeline, param_grid = {})
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [9]:
# Perform predictions using the model
pred = cv.predict(X_test)
print(pred)

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  1.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
print(classification_report(y_test.values, pred, target_names = Y.columns.values))

                        precision    recall  f1-score   support

               related       0.85      0.81      0.83      1662
               request       0.80      0.67      0.73       930
                 offer       0.00      0.00      0.00         3
           aid_related       0.82      0.64      0.72      1021
          medical_help       0.43      0.02      0.04       137
      medical_products       0.67      0.04      0.08        96
     search_and_rescue       0.00      0.00      0.00        57
              security       0.00      0.00      0.00        30
              military       0.00      0.00      0.00        10
           child_alone       0.00      0.00      0.00         0
                 water       0.92      0.48      0.63       211
                  food       0.91      0.66      0.77       410
               shelter       0.85      0.30      0.45       299
              clothing       0.00      0.00      0.00        22
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
parameters = {
    'clf__max_depth': [None, 10, 15, 12],
    'clf__n_estimators': [10, 15]
}

cv_g = GridSearchCV(pipeline, param_grid = parameters)
cv_g.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__max_depth': [None, 10, 15, 12], 'clf__n_estimators': [10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
pred = cv_g.predict(X_test)

In [15]:
print(classification_report(y_test.values, pred, target_names = Y.columns.values))

                        precision    recall  f1-score   support

               related       0.85      0.81      0.83      1638
               request       0.81      0.60      0.69       889
                 offer       0.00      0.00      0.00         1
           aid_related       0.84      0.59      0.69       997
          medical_help       0.75      0.02      0.04       142
      medical_products       0.75      0.04      0.08        72
     search_and_rescue       0.00      0.00      0.00        38
              security       0.00      0.00      0.00        36
              military       0.00      0.00      0.00         7
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.52      0.65       198
                  food       0.90      0.61      0.73       375
               shelter       0.87      0.33      0.48       251
              clothing       0.00      0.00      0.00        25
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
Y = Y.drop(['child_alone'], axis = 1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

### 9. Export your model as a pickle file

In [14]:
pipeline_2 = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)), 
                    ('tfidf', TfidfTransformer()),
                    ('clfa', MultiOutputClassifier(SVC()))])

In [17]:
parameters = {
    'clfa__estimator__C': [0.1, 1, 10],
    'clfa__estimator__kernel': ['rbf', 'linear']
}

cv_a = GridSearchCV(pipeline_2, param_grid = parameters)
cv_a.fit(X_train, y_train)
pred_a = cv_a.predict(X_test)
print(classification_report(y_test.values, pred_a, target_names = Y.columns.values))

                        precision    recall  f1-score   support

               related       0.85      0.86      0.86      1652
               request       0.78      0.75      0.76       862
                 offer       0.00      0.00      0.00         4
           aid_related       0.77      0.75      0.76       934
          medical_help       0.66      0.13      0.22       144
      medical_products       0.86      0.23      0.37        82
     search_and_rescue       0.00      0.00      0.00        52
              security       0.00      0.00      0.00        32
              military       0.00      0.00      0.00        11
                 water       0.88      0.88      0.88       209
                  food       0.88      0.91      0.90       367
               shelter       0.76      0.66      0.71       237
              clothing       0.67      0.24      0.35        17
                 money       0.00      0.00      0.00        39
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [20]:
import joblib
joblib.dump(cv_g, 'classifier.pkl', compress = True)

['classifier.pkl']

In [21]:
joblib.dump(cv_a, 'classifier.pkl', compress = True)

['classifier.pkl']